In [1]:
import pandas as pd
import json
import numpy as np

# df = pd.read_csv("data\part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv")
df = pd.read_csv("data/part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv")
df.rename(columns={'Magnitue': 'Magnitude'}, inplace=True)

# selected_labels = [
#     'BenignTraffic', 'BrowserHijacking', 'CommandInjection',
#     'DDoS-ICMP_Fragmentation', 'DNS_Spoofing', 'MITM-ArpSpoofing',
#     'Recon-OSScan', 'VulnerabilityScan'
# ]

selected_labels = [
    # 'BenignTraffic', 'DDoS-ICMP_Fragmentation', 'DNS_Spoofing', 'MITM-ArpSpoofing', 'Recon-OSScan', 'VulnerabilityScan'
    'BenignTraffic', 'DDoS-ICMP_Fragmentation', 'DNS_Spoofing', 'MITM-ArpSpoofing', 'Recon-OSScan', 'Mirai-udpplain'
]
df = df[df['label'].isin(selected_labels)]
print(df['label'].unique())

print(df.columns)

# Benign Traffic, Browser Hijacking, Command injection, DDoS-ICMP_Fragmentation, DNS Spoofing, MITM-ArpSpoofing, Recon-OSScan, Vulnerability scan.
# Benign Traffic, Command injection, DDoS-ICMP_Fragmentation, DNS Spoofing, MITM-ArpSpoofing, Recon-OSScan, Vulnerability scan.

['Mirai-udpplain' 'BenignTraffic' 'DNS_Spoofing' 'MITM-ArpSpoofing'
 'Recon-OSScan' 'DDoS-ICMP_Fragmentation']
Index(['flow_duration', 'Header_Length', 'Protocol Type', 'Duration', 'Rate',
       'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count', 'syn_count',
       'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet',
       'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC',
       'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number',
       'Magnitude', 'Radius', 'Covariance', 'Variance', 'Weight', 'label'],
      dtype='object')


In [2]:
# Select only 'label' and 'Magnitude' columns
selected_data = df[['label', 'Magnitude']]

# Group by 'label' and aggregate 'Magnitude' values into a list
grouped_data = selected_data.groupby('label')['Magnitude'].agg(list).reset_index()

# Save the selected data to a JSON file
grouped_data.to_json('JSON/boxplot_data.json', orient='records', lines=False)

In [3]:
# Group the DataFrame by 'label' and convert each group to a list
grouped_data = df.groupby('label')['Duration'].apply(list)

# Convert the grouped data to a list of dictionaries
data = [{'label': label, 'Duration': durations} for label, durations in grouped_data.items()]

# Write the Python object to a file
with open('JSON/violin-plot_Duration_data.json', 'w') as f:
    json.dump(data, f, indent=4)

In [4]:
selected_columns = ['Duration', 'Tot size', 'label']
grouped_data = df[selected_columns]

# Group by 'label' and aggregate 'Duration' and 'Tot size' values into a list
selected_data = grouped_data.groupby('label').agg(list).reset_index()

# Keep only the first 10 values for 'Duration' and 'Protocol Type' for every label
selected_data['Duration'] = selected_data['Duration'].apply(lambda x: x[:20])
selected_data['Tot size'] = selected_data['Tot size'].apply(lambda x: x[:20])

# Convert DataFrame to JSON string
json_str = selected_data.to_json(orient='records')

# Parse JSON string to Python object
data = json.loads(json_str)

# Write pretty-printed JSON to file
with open('JSON/scatter-plot_Size_data.json', 'w') as f:
    json.dump(data, f, indent=4)

In [5]:
# Step 1: Select numerical columns
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()

# Remove "IAT" and "Tot sum" from the list of numerical columns
numerical_cols = [col for col in numerical_cols if col not in ["IAT", "Tot sum"]]

# Step 2: Calculate variance for each numerical column (excluding "IAT") and select the top 10
top_variance_cols = df[numerical_cols].var().nlargest(10).index.tolist()

# Step 3: Print the top 10 columns with the highest numeric variance
print("Top 10 Columns with Highest Numeric Variance (excluding 'IAT' and 'Tot sum'):")
print(", ".join(top_variance_cols))

# Save the normalized data to a JSON file
df[top_variance_cols].to_json("JSON/spider-plot_normalized_data.json", orient="index", indent=4)


Top 10 Columns with Highest Numeric Variance (excluding 'IAT' and 'Tot sum'):
Header_Length, Covariance, Rate, Srate, Max, rst_count, flow_duration, Radius, Tot size, AVG


In [6]:
# selected_columns = ['label', 'Rate', 'Duration']
# grouped_data = df[selected_columns]

# # Group by 'label' and aggregate 'Duration' and 'Rate' values into lists
# selected_data = grouped_data.groupby('label').agg(list).reset_index()

# # Adjust 'Rate' values based on thresholds
# selected_data['Rate'] = selected_data['Rate'].apply(lambda x: [val / 10 if val > 100 else val for val in x])
# selected_data['Rate'] = selected_data['Rate'].apply(lambda x: [val / 100 if val > 1000 else val for val in x])

# # Ensure 'Rate' values are not greater than 200
# selected_data['Rate'] = selected_data['Rate'].apply(lambda x: [min(val, 200) for val in x])

# # Reduce the number of values for 'Duration' and 'Rate' for every label
# selected_data['Duration'] = selected_data['Duration'].apply(lambda x: x[:15])
# selected_data['Rate'] = selected_data['Rate'].apply(lambda x: x[:15])

# # Sort the values of the columns in ascending order
# selected_data['Duration'] = sorted(selected_data['Duration'])
# selected_data['Rate'] = sorted(selected_data['Rate'])

# # Convert DataFrame to JSON string
# json_str = selected_data.to_json(orient='records')

# # Parse JSON string to Python object
# data = json.loads(json_str)

# # Write pretty-printed JSON to file
# with open('JSON/line_plot_data.json', 'w') as f:
#     json.dump(data, f, indent=4)

In [7]:
import json
import pandas as pd

def normalize_value(x):
    while x > 999:
        x /= 10
    return x

selected_columns = ['label', 'Header_Length', 'Duration']
grouped_data = df[selected_columns]

# Group by 'label' and aggregate 'Duration' and 'Header_Length' values into lists
selected_data = grouped_data.groupby('label').agg(list).reset_index()

# Reduce the number of values for 'Duration' and 'Header_Length' for every label
selected_data['Duration'] = selected_data['Duration'].apply(lambda x: [normalize_value(i) for i in x[:15]])
selected_data['Header_Length'] = selected_data['Header_Length'].apply(lambda x: [normalize_value(i) for i in x[:15]])

# Sort the values of 'Duration' and 'Header_Length' columns in ascending order
selected_data['Duration'] = selected_data['Duration'].apply(lambda x: sorted(x))
selected_data['Header_Length'] = selected_data['Header_Length'].apply(lambda x: sorted(x))

# Convert DataFrame to a list of dictionaries
records = selected_data.to_dict(orient='records')

# Convert the list of dictionaries to JSON
json_str = json.dumps(records, indent=4)

# Write pretty-printed JSON to file
with open('JSON/line_plot_data.json', 'w') as f:
    f.write(json_str)